#### Import necessary dependecies

In [ ]:
import pandas as pd
import geopandas as gpd
import folium

#### Load the CSV file with municipal uptake rate

In [ ]:
df_csv = pd.read_csv("https://2207-17-fibre-competitive-intensity-model-b.s3.eu-west-1.amazonaws.com/Preprocessed+Data/SA-municipality-uptake-rate.csv")

#### Display the first few rows of the DataFrame

In [ ]:
df_csv.head()

#### Load the shapefile of South Africa municipalities

In [ ]:
gdf_municipalities = gpd.read_file("https://2207-17-fibre-competitive-intensity-model-b.s3.eu-west-1.amazonaws.com/MDB_Local_Municipal_Boundary_2018.geojson")

#### Display the first few rows of the DataFrame

In [ ]:
gdf_municipalities.head()

#### Convert the entries in the 'MUNICNAME' column to lowercase

In [ ]:
gdf_municipalities['MUNICNAME'] = gdf_municipalities['MUNICNAME'].str.lower()

#### Rename the 'MUNICNAME' column

In [ ]:
gdf_municipalities = gdf_municipalities.rename(columns={
    'MUNICNAME': 'municipality'
})

#### Display the first few rows of the DataFrame

In [ ]:
gdf_municipalities.head()

#### Merge the two DataFrames based on the municipality name

In [ ]:
merged_df = pd.merge(gdf_municipalities, df_csv, on="municipality")

#### Display the first few rows of the DataFrame

In [ ]:
merged_df.head()

#### Convert the DATE column to a string(Object of type Timestamp should be JSON serializable for Folium to run)

In [ ]:
merged_df['DATE'] = merged_df['DATE'].astype(str)

#### Convert the merged DataFrame into a GeoJSON file

In [ ]:
merged_geojson = merged_df.to_crs(epsg='4326').to_json()

#### Create the Folium map

In [ ]:
m = folium.Map(location=[-29.0562, 25.4377], zoom_start=5)

#### Add the choropleth layer to the map

In [ ]:
folium.Choropleth(
    geo_data=merged_geojson,
    data=merged_df,
    columns=['municipality', 'uptake_rate_pop'],
    key_on='feature.properties.municipality',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Fibre Uptake Rate_pop (%)'
).add_to(m)

#### Add GeoJsonTooltip to show the exact uptake rate

In [ ]:
folium.GeoJson(
    merged_geojson,
    name='tooltip',
    tooltip=folium.features.GeoJsonTooltip(
        fields=['municipality', 'uptake_rate_pop'],
        aliases=['Municipality:', 'Uptake Rate:'],
        style=('background-color: grey; color: white;')
    )
).add_to(m)

#### Display the map

In [ ]:
m